### Credits:

## Problem 3.2: Pulsing jellyfish (50 pts)

In [Tutorial 3b](../tutorials/t3b_extracting_info_from_images.html), we extracted information from movies of pulsing jellyfish. Using these images (which you can download [here](../data/cassiopea_pulsation.zip), analyze both the daytime and nighttime movies. Compare and contrast the measurements both between daytime and nighttime and among the individual jellyfish. Discuss any conclusions or suggestions for further experiments you may have.

*This question is intentionally open-ended. Think carefully about what you want to do and how you interpret what you see.*

## Solution

In [1]:
import glob
import os

import numpy as np
import pandas as pd
import scipy.signal

# Image processing tools
import skimage
import skimage.io

import bebi103
import bokeh
bokeh.io.output_notebook()

Loading BokehJS ...

The first step to analyzing this data is loading it, defining ROIs, and plotting the frequencies. My annotations of this process will be sparse because it is taken verbatum from tutorial 3b. 

First I will get filenames for image data.

In [2]:
# The directory containing daytime data
data_dir_day = '../data/Cassiopea_Pulsation/day'
data_dir_night = '../data/Cassiopea_Pulsation/night'

# Glob string for images
im_glob_day = os.path.join(data_dir_day, '*.TIF')
im_glob_night = os.path.join(data_dir_night, '*.TIF')

# Get list of files in directory
im_list_day = sorted(glob.glob(im_glob_day))
im_list_night = sorted(glob.glob(im_glob_night))

# Check to make sure we have the right filenames.
im_list_night[:5]

['../data/Cassiopea_Pulsation/night/Frame_545000.TIF',
 '../data/Cassiopea_Pulsation/night/Frame_545001.TIF',
 '../data/Cassiopea_Pulsation/night/Frame_545002.TIF',
 '../data/Cassiopea_Pulsation/night/Frame_545003.TIF',
 '../data/Cassiopea_Pulsation/night/Frame_545004.TIF']

The filenames look good. Brief validation that the images are greyscale was performed in the tutorial, so I will assume that here, and efficiently load the images as an image collection using sci-kit image and tutorial code. 

In [3]:
def squish_rgb(fname, **kwargs):
    """
    Only take one channel. (Need to explicitly have the **kwargs to play
    nicely with skimage.io.ImageCollection.)
    """
    im = skimage.io.imread(fname)
    return im[:,:,0]

# Load in day images
ic_day = skimage.io.ImageCollection(im_glob_day,
                                    conserve_memory=True, 
                                    load_func=squish_rgb)

# Load in night images
ic_night = skimage.io.ImageCollection(im_glob_night,
                                      conserve_memory=True, 
                                      load_func=squish_rgb)

I will also make time series for both image collections, assuming again that the rate is 15 frames per second. 

In [4]:
fps = 15
t_day = np.arange(0, len(ic_day)) / fps
t_night = np.arange(0, len(ic_night)) / fps
t_night, t_day

(array([0.00000000e+00, 6.66666667e-02, 1.33333333e-01, ...,
        2.66466667e+02, 2.66533333e+02, 2.66600000e+02]),
 array([0.00000000e+00, 6.66666667e-02, 1.33333333e-01, ...,
        2.66466667e+02, 2.66533333e+02, 2.66600000e+02]))

Looks like both datasets are about 267 seconds long, which seems reasonable. 

Dr. Bois said that the jellyfish will not shift during the experiment, for which we are very thankful. Thus, we will draw a set of ROIs for each image collection, one ROI per jellyfish. 

The following code checks to see if there is already an ROI file, and prompts you to draw rois if there is not. Otherwise, it simply loads the ROIs into a dataframe. 

In [5]:
df_rois = None
try:
    df_rois = pd.read_csv("./roi_dataframe.csv")
    df_rois = df_rois.drop(["Unnamed: 0"], axis = 1)
except:
    print("ROIs not found. Please draw ROIs, or place them in the appropriate directory.")
    # Read in the first images using skimage
    im_day = skimage.io.imread(im_list_day[0])

    # Draw day ROIs
    print("Please draw day ROIs.")
    rois_day = bebi103.viz.draw_rois(im_day, flip=False)

Now to draw night ROIs, if not already drawn. 

In [6]:
if type(df_rois) == type(None):
    print("Please draw night ROIs before executing the next cell.")
    im_night = skimage.io.imread(im_list_day[0])
    rois_night = bebi103.viz.draw_rois(im_night, flip=False)

Looks great! Now it is time to add night/day fields and save the ROIs as a csv, so that you never ever ever ever have to draw them again. 

In [7]:
if type(df_rois) == type(None):
    # Turn the ROIs into dataframes
    df_rois_day = bebi103.viz.roicds_to_df(rois_day)
    df_rois_night = bebi103.viz.roicds_to_df(rois_night)
    
    # Add a day or a night column
    daytime_arr = ["day"] * len(df_rois_day.index)
    nighttime_arr = ["night"] * len(df_rois_night.index)
    df_rois_day["time_of_day"] = daytime_arr
    df_rois_night["time_of_day"] = nighttime_arr
    
    # Save ROIs as a .csv
    df_rois = pd.concat([df_rois_day, df_rois_night])
    df_rois.to_csv(path_or_buf="./roi_dataframe.csv")

In [12]:
df_rois.head(100)

,roi,x,y,time_of_day
0,0,545.587583,331.660714,day
1,0,579.534368,318.701786,day
2,0,588.575388,279.182143,day
3,0,580.260532,244.328571,day
4,0,512.400222,246.358929,day
5,0,508.370288,322.103571,day
6,0,525.000000,328.526786,day
7,1,414.478936,357.198214,day
8,1,428.342572,366.583929,day
9,1,462.233925,367.789286,day


In [19]:
im_night = skimage.io.imread(im_list_night[0])
im_day = skimage.io.imread(im_list_day[0])

# Manually reduce image dimensionality. 
im_day = im_day[:,:,0]
im_night = im_night[:,:,0]

rois_day = [bebi103.image.verts_to_roi(g[['x', 'y']].values, *im_day.shape)
            for _, g in df_rois[df_rois['time_of_day']=='day'].groupby('roi')]

rois_night = [bebi103.image.verts_to_roi(g[['x', 'y']].values, *im_night.shape)
            for _, g in df_rois[df_rois['time_of_day']=='night'].groupby('roi')]